In [8]:
import pygame
import sys
import time
import os
import random
import pickle



class Thirst:
    def __init__(self, initial_thirst=50, ticket=None):
        self.thirst = initial_thirst
        self.ticket = ticket

    def water(self):
        if self.ticket.ticket_thirst > 0:
            if self.thirst >= 10:
                self.thirst -= 10
                self.ticket.ticket_thirst -= 1
            elif 0 <= self.thirst < 10:
                self.thirst = 0
                self.ticket.ticket_thirst -= 1
            return True
        else:
            return False

    def __str__(self):
        return f"{self.thirst}"

class Hunger:
    def __init__(self, initial_hunger=50, ticket=None):
        self.hunger = initial_hunger
        self.ticket = ticket

    def feed(self):
        if self.ticket.ticket_hunger > 0:
            if self.hunger >= 10:
                self.hunger -= 10
                self.ticket.ticket_hunger -= 1
            elif 0 <= self.hunger < 10:
                self.hunger = 0
                self.ticket.ticket_hunger -= 1
            return True
        else:
            return False

    def __str__(self):
        return f"{self.hunger}"

class Relation:
    def __init__(self, initial_relation=0, ticket=None):
        self.relation = initial_relation
        self.ticket = ticket

    def joy(self):
        if self.ticket.ticket_relation > 0:
            if self.relation < 100:
                self.relation += 5
                self.ticket.ticket_relation -= 1
            return True
        else:
            return False

    def __str__(self):
        return f"{self.relation}"

class Level:
    def __init__(self, initial_level=1, asset_loader=None):
        self.level = initial_level
        self.asset_loader = asset_loader 
        if self.asset_loader:
            self.asset_loader.load_assets() 
        self.update_sqr()

    def level_up(self):
        self.level += 1
        self.update_sqr() 

    def update_sqr(self):
        if self.asset_loader:
            new_size = self.calculate_new_size()
            self.asset_loader.SQR = pygame.transform.scale(self.asset_loader.SQR, new_size)

    def calculate_new_size(self):
        size_multiplier = 0.9 + 0.1 * self.level
        width = int(size_multiplier * 250)
        height = int(size_multiplier * 250)
        return (width, height)

    def __str__(self):
        return f"{self.level}"

    def __getstate__(self):
        state = self.__dict__.copy()
        state['asset_loader'] = None 
        return state

    def __setstate__(self, state):
        self.__dict__.update(state)
        self.asset_loader = None 

class Ticket:
    def __init__(self, initial_ticket=0):
        self.ticket_thirst = initial_ticket
        self.ticket_hunger = initial_ticket
        self.ticket_relation = initial_ticket

    def earning(self):
        self.ticket_thirst += random.randint(1, 3)
        self.ticket_hunger += random.randint(1, 3)
        self.ticket_relation += random.randint(1, 3)

    def __str__(self):
        return f"물주기 티켓 : {self.ticket_thirst} | 밥주기 티켓 : {self.ticket_hunger} | 놀아주기 티켓 : {self.ticket_relation}"

class Quiz:
    def __init__(self):
        self.questions = [
            "하늘다람쥐는 주로 낮에 활동하며, 밤에는 둥지에서 휴식을 취한다.",
            "하늘다람쥐의 주요 먹이는 나무의 잎과 작은 곤충들이다.",
            "하늘다람쥐는 날지 못하지만, 긴 거리의 활강을 할 수 있다.",
            "하늘다람쥐는 북반구의 온대와 한대 지역에서만 서식한다.",
            "하늘다람쥐는 현재 IUCN 적색목록에서 '멸종위기' 등급으로 분류된다.",
            "지구 온난화는 현재 인간의 활동에 의해 주로 발생하는 현상이다.",
            "생물다양성 손실은 주로 자연재해와 기후 변화와 같은 자연적인 요인으로 인해 발생한다.",
            "오염된 물은 주로 해양 생태계에만 영향을 미치며, 육지 생태계에는 큰 영향을 미치지 않는다.",
            "국제적인 환경 보호 협약은 주로 각국의 경제적 이익을 위해 개발되며, 실질적인 환경 보호에 큰 도움이 되지 않는다.",
            "태평양의 마리아나 해구는 지구에서 가장 깊은 곳으로 알려져 있다.",
            "쓰레기가 바다로 유출되면 해양 생태계에 심각한 피해를 줄 수 있다.",
            "산불은 기후 변화의 한 예로서, 기후 변화로 인해 더 많은 산불이 발생한다.",
            "세계적으로 심각한 공기 오염으로 인해 사람들의 건강에 악영향을 미치고 있다.",
            "탄소 배출량을 줄이기 위한 정책과 대안들이 전 세계적으로 채택되고 있다.",
            "탄소 중립은 탄소 배출과 탄소 흡수가 서로 상쇄되는 상태를 의미한다.",
            "오존층 파괴는 자외선 차단 효과를 약화시켜 인간과 환경에 해를 끼칠 수 있다.",
            "침수는 지구 온난화로 인해 해수면이 상승함에 따라 발생하는 자연재해 중 하나이다.",
            "유전자 수정 식품은 농업 생산성을 높이고 식량 부족 문제를 해결하는 데 도움이 될 수 있다.",
            "식물 다양성은 생태계 안정성과 기능을 유지하는 데 중요한 역할을 한다.",
            "인공 지능 기술은 환경 모니터링과 보호에 적극적으로 활용되고 있다.",
            "지구 온난화로 인한 극지방의 얼음 녹은 양이 증가하고 있다.",
            "재생 가능 에너지는 환경 보호에 중요한 역할을 하며 지속 가능한 에너지 공급의 핵심이다.",
            "도시화는 자원 소비와 환경 파괴에 영향을 미치는 지속 가능하지 않은 경향이 있다.",
            "산업 폐기물 처리는 지구 환경에 미치는 부정적인 영향을 줄이기 위해 중요한 과제이다.",
            "기후 변화에 대한 대응을 위해 국제적인 협력과 조치가 필요하다.",
            "미세 플라스틱은 해양 생태계에 치명적인 영향을 미치며, 생태계의 미래에 위협을 가한다.",
            "친환경 제품과 서비스의 수요가 증가하고 있다.",
            "생태 관광은 지속 가능한 여행과 자연 보호를 강조하는 산업으로 성장하고 있다.",
            "국제적인 환경 보호 단체와 운동은 지구 환경 문제에 대한 인식과 대응을 촉진한다.",
            "재활용은 자원 절약과 환경 보호에 기여하는 중요한 활동이다."
        ]
        self.answers = [False, False, True, True, False, True, False, False, False, True, True, True, True,
                        True, False, True, True, False, True, True, True, True, True, True, False, True, True, True, True, True]
        self.index = 0

    def get_current_question(self):
        return self.questions[self.index]

    def check_answer(self, put):
        correct = put == self.answers[self.index]
        self.index += 1
        if self.index >= len(self.questions):
            self.index = 0  
        return correct

class Notifier:
    def __init__(self):
        self.message = ""

    def notify(self, message):
        self.message = message

    def clear(self):
        self.message = ""

    def __str__(self):
        return self.message


class AssetLoader:
    def __init__(self):
        self.SQR = None
        self.MAIN_SCREEN = None
        self.sound = None

    def load_assets(self):

        self.yes = pygame.image.load("ape/yes.png")
        self.yes = pygame.transform.scale(self.yes, (150, 150))
        self.yes_rect = self.yes.get_rect(topleft=(50, 300))

        self.quiz_pan = pygame.image.load("ape/quiz_pan.png")

        self.no = pygame.image.load("ape/no.png")
        self.no = pygame.transform.scale(self.no, (150, 150))
        self.no_rect = self.no.get_rect(topleft=(500, 300))
        
        self.SQR = pygame.image.load("ape/sqr1.png")
        self.SQR = pygame.transform.scale(self.SQR, (250, 250))
        
        self.MAIN_SCREEN = pygame.image.load("ape/main_screen.png")

        self.background = pygame.image.load("ape/background.jpg")
        self.background = pygame.transform.scale(self.background, (700, 500))
        
        self.status = pygame.image.load("ape/status.png")
        
        self.logo = pygame.image.load("ape/logo.png")
        self.logo = pygame.transform.scale(self.logo, (300, 120))
        self.logo_rect = self.logo.get_rect(topleft=(200, 0))
        
        self.start = pygame.image.load("ape/start.png")
        self.start_rect = self.start.get_rect(topleft=(200, 300))
        
        self.feed_button = pygame.image.load("ape/feed.png")
        self.feed_button = pygame.transform.scale(self.feed_button, (120, 90))
        self.feed_button_rect = self.feed_button.get_rect(topleft=(350, 380))
        
        self.water_button = pygame.image.load("ape/water.png")
        self.water_button = pygame.transform.scale(self.water_button, (120, 90))
        self.water_button_rect = self.water_button.get_rect(topleft=(500, 380))
        
        self.joy_button = pygame.image.load("ape/joy.png")
        self.joy_button = pygame.transform.scale(self.joy_button, (120, 90))
        self.joy_button_rect = self.joy_button.get_rect(topleft=(200, 380))
        
        self.quiz_button = pygame.image.load("ape/quiz.png")
        self.quiz_button = pygame.transform.scale(self.quiz_button, (120, 90))
        self.quiz_button_rect = self.quiz_button.get_rect(topleft=(50, 380))
        
        self.save_button = pygame.image.load("ape/save.png")
        self.save_button = pygame.transform.scale(self.save_button, (40, 40))
        self.save_button_rect = self.save_button.get_rect(topleft=(550, 0))
        
        self.load_button = pygame.image.load("ape/load.png")
        self.load_button = pygame.transform.scale(self.load_button, (40, 40))
        self.load_button_rect = self.load_button.get_rect(topleft=(600, 0))
        
        self.exit_button = pygame.image.load("ape/exit.png")
        self.exit_button = pygame.transform.scale(self.exit_button, (40, 40))
        self.exit_button_rect = self.exit_button.get_rect(topleft=(650, 0))
        
        self.mouse_image = pygame.image.load('pointer.svg')
        self.load_sound()

    def load_sound(self):
        sound_file = "ape/bgm.mp3"
        if os.path.exists(sound_file):
            pygame.mixer.init()
            self.sound = pygame.mixer.Sound(sound_file)

class ScreenManager:
    def __init__(self, screen, text, clock, text_big):
        self.screen = screen
        self.text = text
        self.clock = clock
        self.text_big = text_big

    def render_text(self, word, color=(0, 0, 0)):
        return self.text.render(word, True, color)

    def render_text_big(self, word, color=(0, 0, 0)):
        return self.text_big.render(word, True, color)

    def draw(self, asset_loader, game_state):
        self.screen.fill((217, 217, 217))
        self.clock.tick(30)
        
        if game_state.page == 1:
            self.screen.blit(asset_loader.MAIN_SCREEN, (0, 0))
            self.screen.blit(asset_loader.logo, asset_loader.logo_rect.topleft)
            self.screen.blit(asset_loader.start, asset_loader.start_rect.topleft)
            
        elif game_state.page == 2:
            self.screen.blit(asset_loader.background, (0, 0))
            self.screen.blit(asset_loader.SQR, (220, 120))
            self.screen.blit(asset_loader.status, (0, 0))
            self.screen.blit(asset_loader.save_button, asset_loader.save_button_rect.topleft)
            self.screen.blit(asset_loader.load_button, asset_loader.load_button_rect.topleft)
            self.screen.blit(asset_loader.exit_button, asset_loader.exit_button_rect.topleft)
            self.screen.blit(asset_loader.quiz_button, asset_loader.quiz_button_rect.topleft)
            self.screen.blit(asset_loader.joy_button, asset_loader.joy_button_rect.topleft)
            self.screen.blit(asset_loader.feed_button, asset_loader.feed_button_rect.topleft)
            self.screen.blit(asset_loader.water_button, asset_loader.water_button_rect.topleft)
            self.screen.blit(self.render_text(str(game_state.thirst)), (190, 28))
            self.screen.blit(self.render_text(str(game_state.hunger)), (190, 63))
            self.screen.blit(self.render_text(str(game_state.relation)), (190, 98))
            self.screen.blit(self.render_text(str(game_state.level)), (70, 55))
            self.screen.blit(self.render_text(str(game_state.notifier)), (235, 150))
            self.screen.blit(self.render_text('물주기 티켓 ' + str(game_state.ticket.ticket_thirst)), (530, 60))
            self.screen.blit(self.render_text('밥주기 티켓 ' + str(game_state.ticket.ticket_hunger)), (530, 90))
            self.screen.blit(self.render_text('놀아주기 티켓 ' + str(game_state.ticket.ticket_relation)), (510, 120))
            game_state.notifier.clear()

        elif game_state.page == 3:
            current_quiz = game_state.quiz.get_current_question()
            self.screen.blit(asset_loader.quiz_pan, (0, 0))
            self.screen.blit(asset_loader.yes, asset_loader.yes_rect.topleft)
            self.screen.blit(asset_loader.no, asset_loader.no_rect.topleft)
            self.screen.blit(self.render_text_big(current_quiz), (20, 100)) 
            

        self.mouse_pos = pygame.mouse.get_pos()
        self.screen.blit(asset_loader.mouse_image, self.mouse_pos)


class EventManager:
    def handle_events(self, game_state, asset_loader):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_state.running = game_state.logout()

            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = event.pos
                if game_state.page == 1:
                    if asset_loader.start_rect.collidepoint(mouse_pos):
                        game_state.login()
                elif game_state.page == 2:
                    if asset_loader.feed_button_rect.collidepoint(mouse_pos):
                        if game_state.hunger.feed():
                            game_state.notifier.notify('배고픔이 해소됩니다')
                        game_state.check_level_up()
                    elif asset_loader.water_button_rect.collidepoint(mouse_pos):
                        if game_state.thirst.water():
                            game_state.notifier.notify('목마름이 해소됩니다')
                        game_state.check_level_up()
                    elif asset_loader.joy_button_rect.collidepoint(mouse_pos):
                        if game_state.relation.joy():
                            game_state.notifier.notify('친밀도가 올라갑니다')
                        game_state.check_level_up()
                    elif asset_loader.quiz_button_rect.collidepoint(mouse_pos):
                        game_state.page = 3
                        game_state.ticket.earning()
                        game_state.update_tickets()
                        game_state.notifier.notify('티켓을 얻습니다')
                    elif asset_loader.save_button_rect.collidepoint(mouse_pos):
                        game_state.save_game()
                        game_state.notifier.notify('게임이 저장되었습니다')
                    elif asset_loader.load_button_rect.collidepoint(mouse_pos):
                        game_state.load_game()
                        game_state.notifier.notify('게임이 로드되었습니다')
                    elif asset_loader.exit_button_rect.collidepoint(mouse_pos):
                        game_state.running = game_state.logout()
                   
                elif game_state.page == 3:
                    if asset_loader.yes_rect.collidepoint(mouse_pos):
                        self.process_answer(game_state, True)
                    elif asset_loader.no_rect.collidepoint(mouse_pos):
                        self.process_answer(game_state, False)

    def process_answer(self, game_state, put):
        correct = game_state.quiz.check_answer(put)
        if correct:
            game_state.thirst.ticket.earning()
            game_state.hunger.ticket.earning()
            game_state.relation.ticket.earning()
        game_state.page = 2
                    

class SoundManager:
    def __init__(self):
        self.sound_playing = False

    def play_sound(self, asset_loader):
        if not self.sound_playing:
            asset_loader.sound.play()
            self.sound_playing = True
        if not pygame.mixer.get_busy():
            self.sound_playing = False

class GameState:
    def __init__(self, asset_loader):
        self.asset_loader = asset_loader 
        self.quiz = Quiz()  
        self.ticket = Ticket()
        self.ticket_thirst = self.ticket.ticket_thirst
        self.ticket_hunger = self.ticket.ticket_hunger
        self.ticket_relation = self.ticket.ticket_relation
        self.thirst = Thirst(ticket=self.ticket)
        self.hunger = Hunger(ticket=self.ticket)
        self.relation = Relation(ticket=self.ticket)
        self.level = Level(asset_loader=self.asset_loader) 
        self.notifier = Notifier()
        self.page = 1
        self.running = True

    def update_tickets(self):
        self.ticket_thirst = self.ticket.ticket_thirst
        self.ticket_hunger = self.ticket.ticket_hunger
        self.ticket_relation = self.ticket.ticket_relation

    def logout(self):
        return False

    def login(self):
        self.page = 2

    def save_game(self):
        game_state = {
            "ticket": self.ticket,
            "thirst": self.thirst,
            "hunger": self.hunger,
            "relation": self.relation,
            "level": self.level,
        }
        with open('game_save.pkl', 'wb') as f:
            pickle.dump(game_state, f)

    def load_game(self):
        if os.path.exists('game_save.pkl'):
            with open('game_save.pkl', 'rb') as f:
                game_state = pickle.load(f)
                self.ticket = game_state["ticket"]
                self.thirst = game_state["thirst"]
                self.hunger = game_state["hunger"]
                self.relation = game_state["relation"]
                self.level = game_state["level"]
                self.level.asset_loader = self.asset_loader 
                self.level.update_sqr()  
                self.update_tickets()

    def check_level_up(self):
        if self.hunger.hunger == 0 and self.thirst.thirst == 0 and self.relation.relation >= 100:
            self.level.level_up()
            self.notifier.notify('레벨이 오릅니다')
            self.hunger.hunger = 100
            self.thirst.thirst = 100
            self.relation.relation = 0

class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption('ANIMAL PROTECT EARTH')
        self.MAX_WIDTH = 700
        self.MAX_HEIGHT = 500
        self.screen = pygame.display.set_mode((self.MAX_WIDTH, self.MAX_HEIGHT))
        self.text = pygame.font.SysFont('hy중고딕', 20)
        self.text_big = pygame.font.SysFont('hy중고딕', 20)
        self.clock = pygame.time.Clock()

        self.asset_loader = AssetLoader()
        self.asset_loader.load_assets()

        self.screen_manager = ScreenManager(self.screen, self.text, self.clock, self.text_big)
        self.event_manager = EventManager()
        self.sound_manager = SoundManager()

        self.game_state = GameState(asset_loader=self.asset_loader) 

    def run(self):
        while self.game_state.running:
            self.event_manager.handle_events(self.game_state, self.asset_loader)
            self.screen_manager.draw(self.asset_loader, self.game_state)
            self.sound_manager.play_sound(self.asset_loader)
            pygame.display.flip()

if __name__ == "__main__":
    game = Game()
    game.run()
    pygame.quit()
